In [113]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [119]:
from importlib import reload
reload(fm)
reload(dl)
reload(fit)

<module 'fitter.fitter' from '../fitter/fitter.py'>

### Title

In [120]:
params = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt', # 'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'
        'order' : 'n3lo', # 'nlo', 'n2lo', 'n3lo'
        'latt_ct' : 'n4lo',
        'F2' : 'FKFK', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : True,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
    }, 

    'fast_susnet' : True,
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    
    'abbrs' : [u'a06m310L', u'a09m135',  u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
}

In [121]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()
params['model_info']['name'] = data_loader.get_model_name_from_model_info(params['model_info'])


# Make bootstrapper
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=None, **params)

new_prior = fit_manager.create_prior_from_fit()
#data_loader.save_prior(new_prior, bootstrapper.get_name())
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
1   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
2   {'chiral_n2lo': 2.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
3   {'chiral_n2lo': 1.0,'chiral_n3lo': 2.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
4   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
5   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
6   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
7   {'chiral_n2lo': 0.01,'chiral_n3lo': 1.4,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
8   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.6,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
9   {'chiral_n2lo': 0.6,'chiral_n3lo': 0.16,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
10   {'chiral_n2lo': 0.44,'chiral_n3lo': 0.824,'la

78   {'chiral_n2lo': 0.812,'chiral_n3lo': 0.269,'latt_n2lo': 6.45,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
79   {'chiral_n2lo': 0.747,'chiral_n3lo': 0.707,'latt_n2lo': 17.6,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
80   {'chiral_n2lo': 0.798,'chiral_n3lo': 0.862,'latt_n2lo': 9.31,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
81   {'chiral_n2lo': 0.813,'chiral_n3lo': 0.605,'latt_n2lo': 6.6,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
82   {'chiral_n2lo': 0.763,'chiral_n3lo': 0.682,'latt_n2lo': 14.9,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
83   {'chiral_n2lo': 0.797,'chiral_n3lo': 0.63,'latt_n2lo': 9.36,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
84   {'chiral_n2lo': 0.765,'chiral_n3lo': 1.01,'latt_n2lo': 7.61,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
85   {'chiral_n2lo': 0.788,'chiral_n3lo': 0.549,'latt_n2lo': 12.8,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
86   {'chiral_n2lo': 0.758,'chiral_n3lo': 0.418,'latt_n2lo': 14.2,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
87   {'chiral_n2lo': 0.768,'chiral_n3lo'

155   {'chiral_n2lo': 0.0487,'chiral_n3lo': 0.131,'latt_n2lo': 11.0,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
156   {'chiral_n2lo': 0.103,'chiral_n3lo': 0.0386,'latt_n2lo': 10.5,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
157   {'chiral_n2lo': 0.0257,'chiral_n3lo': 0.0168,'latt_n2lo': 11.3,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
158   {'chiral_n2lo': 0.121,'chiral_n3lo': 0.103,'latt_n2lo': 11.1,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
159   {'chiral_n2lo': 0.0473,'chiral_n3lo': 0.0589,'latt_n2lo': 11.0,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
160   {'chiral_n2lo': 0.0697,'chiral_n3lo': 0.216,'latt_n2lo': 11.2,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
161   {'chiral_n2lo': 0.0191,'chiral_n3lo': 0.0441,'latt_n2lo': 11.0,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
162   {'chiral_n2lo': 0.0194,'chiral_n3lo': 0.169,'latt_n2lo': 11.1,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
163   {'chiral_n2lo': 0.0272,'chiral_n3lo': 0.109,'latt_n2lo': 11.3,'latt_n3lo': 1030.0,'latt_n4lo': 1030.0}
164   {'chiral_n2

In [124]:
print(fit_manager)


Model: xpt_FKFK_n3lo_a4_a6_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1937(33) [1.1957(19)]	(delta_su2 = -0.00419(70))

Parameters:
            L_4     -0.00218 (10)       [   0.0000 (50) ]  
            L_5     0.000466 (83)       [   0.0000 (50) ]  
            A_k    0.00002 (1450)       [    0.000 (14) ]  
            A_p   -0.00001 (1450)       [    0.000 (14) ]  
            A_a       -11.0 (1.4)       [        0 (11) ]  
           A_aa           82 (20)       [      0 (1030) ]  
           A_ak      6(27400)e-06       [    0.000 (27) ]  
           A_ap    0.00002 (2740)       [    0.000 (27) ]  
           A_kk    0.00001 (2740)       [    0.000 (27) ]  
           A_kp    1e-06 +- 0.027       [    0.000 (27) ]  
           A_pp     10(27400)e-06       [    0.000 (27) ]  
          A_aaa         43 (1024)       [      0 (1030) ]  

Least Square Fit:
  chi2/dof [dof] = 0.9 [18]    Q = 0.58    logGBF = 71.785

Settings:
  svdcut/n = 1e-12/0    tol = (1e

In [125]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00036
         A_aa:   0.00005
        A_aaa:   0.00041
         A_ak:   0.00000
         A_ap:   0.00001
          A_k:   0.00001
         A_kk:   0.00000
         A_kp:   0.00000
          A_p:   0.00001
         A_pp:   0.00001
          L_4:   0.00003
          L_5:   0.00004
     phys:mpi:   0.00013
      phys:mk:   0.00021
phys:lam2_chi:   0.00127
         stat:   0.00294
------------------------
        total:   0.00326

